In [ ]:
from py2neo import Graph, Node 

In [ ]:
# connect to graph
graph = Graph(uri="localhost", auth=("username", "password"))

In [ ]:
# start a transaction
tx = graph.begin()
# all crud operations below can also be done on graph instead of tx

In [ ]:
# add a new node
# Node(label, property1=value1, property2=value2)
n = Node("Person", name="Alice")
tx.create(n) 

In [ ]:
# add a new node from dict
# Node(label, **dictionary)
person = {"name": "Alice"}
n = Node("Person", **person)
tx.create(n) 

In [ ]:
# upsert node (update if exists else insert)
# equivalence is checked on specified key(s)
# merge(node, label, key)
# merge(node, label, keys_list) 
tx.merge(n, "Person", "name")

In [ ]:
# commit transaction
tx.commit()

In [ ]:
# abort transaction (rollback)
tx.rollback()